<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/Fitbit_V02_Regression_2023_07_03_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import seaborn as sns
import sklearn.metrics as sm


def prim_plot(y_actual,y_pred):
  pred_y_df=pd.DataFrame(
      {
          "Actual_Value":y_actual,
          "Predicted_Value":y_pred,
          "Difference":y_actual-y_pred
      }
  )


  """sns.set(rc={'figure.figsize':(12,10)})
  scatter_poly2 = sns.lmplot(
      data = pred_y_df,
      x = "Actual_Value",
      y = "Predicted_Value",
      order = 2
  )"""

  sns.regplot(x="Actual_Value", y="Predicted_Value", data=pred_y_df)


def res(y_actual,y_pred):
  print("Mean absolute error =", round(sm.mean_absolute_error(y_actual, y_pred), 2))
  print("Mean squared error =", round(sm.mean_squared_error(y_actual, y_pred), 2))
  print("Median absolute error =", round(sm.median_absolute_error(y_actual, y_pred), 2))
  print("Explain variance score =", round(sm.explained_variance_score(y_actual, y_pred), 2))
  print("R2 score =", round(sm.r2_score(y_actual, y_pred), 2))


def sub_wise_splt(df,col,flage):
  total_count = len(set(df[col]))
  subjects=sorted(set(df[col]))

  train_axis = []
  test_axis = []

  count = 0

  for i in subjects:
    if count <= flage:
      train_axis.append(df.query(col+" == '"+i+"'"))
    else:
      test_axis.append(df.query(col+" == '"+i+"'"))
    count+=1

  return pd.concat(train_axis, axis=0).reset_index(drop = True),pd.concat(test_axis, axis=0).reset_index(drop = True)


In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score,mean_squared_error
#dataset=pd.read_csv("/content/drive/MyDrive/Iqram Sir/Fitbit_feature_merged_version_2023.06.20_ST-II&ST-V_Pre.csv")
dataset = pd.read_excel("/content/Fitbit_feature_merged_version_2023.06.29_SPO2_wo_missing column.xlsx")
#dataset=pd.read_excel("/content/drive/MyDrive/Cornell/MADE Lab/Wearable Project/Data/Fitbit ML/Feature merge/Fitbit_feature_merged_version_2023.06.29_SPO2_wo_missing column.xlsx")

target = "DASI - VO2 peak score:"

In [4]:
dataset.head(10)

,Subject,Condition,Activity,Epoch Duration(min),Epoch,nni_counter,nni_mean,nni_min,nni_max,hr_mean,...,Gastrointestinal:.1,Cardiovascular:.1,Neruological:,Wound:.1,Hematological:.1,Pain:.1,Complete?,Has this CRF been quality checked,Date CRF was last checked:,Complete?.1
0,002-JR,post,Light,10,1,69,762.275362,731,810,78.784007,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
1,002-JR,post,Light,10,10,64,752.296875,722,789,79.788063,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
2,002-JR,post,Light,10,100,73,687.876712,638,740,87.334350,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
3,002-JR,post,Light,10,101,78,585.282051,545,645,102.611992,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
4,002-JR,post,Light,10,102,66,603.772727,582,645,99.417782,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
5,002-JR,post,Light,10,103,87,607.206897,555,740,99.222786,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
6,002-JR,post,Light,10,104,69,673.130435,638,740,89.201360,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
7,002-JR,post,Light,10,105,65,653.092308,625,722,92.003173,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
8,002-JR,post,Light,10,106,72,613.986111,576,674,97.842853,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
9,002-JR,post,Light,10,107,75,632.466667,600,714,94.975528,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete


In [5]:
dataset.dropna(axis=0, how='any', inplace = True)
dataset = dataset.reset_index(drop=True)


In [6]:
dataset.shape

(5257, 368)



---



---



#Split Dataset into pre and post

In [7]:
pre_dataset = dataset.query("Condition == 'pre' and Activity == 'Heavy'").reset_index(drop=True)
post_dataset = dataset.query("Condition == 'post' and Activity == 'Heavy'").reset_index(drop=True)


In [ ]:
pre_dataset.head(10)

,Subject,Condition,Activity,Epoch Duration(min),Epoch,nni_counter,nni_mean,nni_min,nni_max,hr_mean,...,Gastrointestinal:.1,Cardiovascular:.1,Neruological:,Wound:.1,Hematological:.1,Pain:.1,Complete?,Has this CRF been quality checked,Date CRF was last checked:,Complete?.1
0,003-ES,pre,Heavy,10,1,66,535.500000,491,571,112.230555,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
1,003-ES,pre,Heavy,10,10,77,571.376623,540,606,105.085716,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
2,003-ES,pre,Heavy,10,11,75,551.400000,512,594,108.930623,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
3,003-ES,pre,Heavy,10,12,74,542.986486,517,582,110.567865,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
4,003-ES,pre,Heavy,10,13,73,548.945205,517,571,109.353778,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
5,003-ES,pre,Heavy,10,14,70,546.085714,500,576,109.981291,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
6,003-ES,pre,Heavy,10,15,85,578.317647,521,659,104.002531,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
7,003-ES,pre,Heavy,10,2,79,570.139241,530,631,105.412330,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
8,003-ES,pre,Heavy,10,3,70,579.385714,526,618,103.711238,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
9,003-ES,pre,Heavy,10,4,84,595.750000,545,645,100.839311,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete


In [ ]:
post_dataset.head(10)

,Subject,Condition,Activity,Epoch Duration(min),Epoch,nni_counter,nni_mean,nni_min,nni_max,hr_mean,...,Gastrointestinal:.1,Cardiovascular:.1,Neruological:,Wound:.1,Hematological:.1,Pain:.1,Complete?,Has this CRF been quality checked,Date CRF was last checked:,Complete?.1
0,003-ES,post,Heavy,10,1,72,553.041667,508,612,108.715940,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
1,003-ES,post,Heavy,10,10,86,615.069767,566,689,97.653615,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
2,003-ES,post,Heavy,10,11,103,592.563107,550,638,101.355242,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
3,003-ES,post,Heavy,10,12,77,641.298701,600,674,93.637935,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
4,003-ES,post,Heavy,10,13,85,580.141176,517,638,103.657284,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
5,003-ES,post,Heavy,10,14,105,563.809524,491,652,106.727549,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
6,003-ES,post,Heavy,10,15,73,567.739726,512,612,105.904177,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
7,003-ES,post,Heavy,10,16,99,593.030303,521,705,101.767119,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
8,003-ES,post,Heavy,10,17,71,570.126761,530,652,105.533158,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
9,003-ES,post,Heavy,10,18,69,555.666667,508,638,108.205167,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete


In [8]:
dataset = pre_dataset

In [9]:
dataset.head()

,Subject,Condition,Activity,Epoch Duration(min),Epoch,nni_counter,nni_mean,nni_min,nni_max,hr_mean,...,Gastrointestinal:.1,Cardiovascular:.1,Neruological:,Wound:.1,Hematological:.1,Pain:.1,Complete?,Has this CRF been quality checked,Date CRF was last checked:,Complete?.1
0,003-ES,pre,Heavy,10,1,66,535.500000,491,571,112.230555,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
1,003-ES,pre,Heavy,10,10,77,571.376623,540,606,105.085716,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
2,003-ES,pre,Heavy,10,11,75,551.400000,512,594,108.930623,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
3,003-ES,pre,Heavy,10,12,74,542.986486,517,582,110.567865,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete
4,003-ES,pre,Heavy,10,13,73,548.945205,517,571,109.353778,...,No,No,No,No,No,No,Complete,Yes,2023-05-15,Complete


#Custom Spliting

In [11]:
len(set(dataset["Subject"]))

13

In [10]:
num_of_train_sub = 15

train,test = sub_wise_splt(dataset,"Subject", num_of_train_sub)

ValueError: ignored

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
X_train = train.iloc[:,6:116]
X_train = X_train.loc[:, X_train.columns != "aEE_MAX(kJ/min)"]

y_train = train[target]

X_test = test.iloc[:,6:116]
X_test = X_test.loc[:, X_test.columns != "aEE_MAX(kJ/min)"]
y_test = test[target]



---



---



In [ ]:
X = dataset.iloc[:,6:116]

In [ ]:
X

In [ ]:
X = dataset.iloc[:,6:116]
X = X.loc[:, X.columns != "aEE_MAX(kJ/min)"]
y = dataset.iloc[:, 184]

In [ ]:
set(X)

In [ ]:
set(y)

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [ ]:
column_name = 'DASI - VO2 peak score:'
column_index = dataset.columns.get_loc(column_name)
column_index

In [ ]:
from sklearn.feature_selection import SelectKBest
bestfeatures = SelectKBest(k=20)
fit = bestfeatures.fit(X,y)


dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']


number_of_feat=int(input("Number Of Best Features: "))


imp=featureScores.nlargest(number_of_feat,'Score')
print(imp)


list_of_feat=[]

for i in range(number_of_feat):
  list_of_feat.append(imp.iloc[:,0:1].values[i][0])


X_new = dataset[list_of_feat]
y_new = dataset.iloc[:, 184]

In [ ]:
X_new.shape

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)




---



In [ ]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt

#Decision Trees

In [ ]:
dtree = DecisionTreeRegressor(max_depth=3, min_samples_leaf=0.003)
dtree.fit(X_train, y_train)

pred_train= dtree.predict(X_train)
print("Trainng Result")
res(y_train,pred_train)

print()

pred_test= dtree.predict(X_test)
print("Trainng Result")
res(y_test,pred_test)

In [ ]:

"""for i in range(1,1000):
  l=i/1000
  print("------------------------------------------------------")
  dtree = DecisionTreeRegressor(max_depth=3, min_samples_leaf=l)
  dtree.fit(X_train, y_train)
  pred_test_tree= dtree.predict(X_test)
  k = r2_score(y_test, pred_test_tree)
  if k>0:
    print(k,l)"""

In [ ]:
prim_plot(y_train,pred_train)

In [ ]:
prim_plot(y_test,pred_test)

#Random Forest (or Bootstrap Aggregation)

In [ ]:
#RF model
model_rf = RandomForestRegressor(n_estimators=700, oob_score=True)
model_rf.fit(X_train, y_train)

print("Training")
pred_train= model_rf.predict(X_train)
res(y_train,pred_train)

print("Testing")
pred_test = model_rf.predict(X_test)
res(y_test,pred_test)

In [ ]:
"""
for i in range(700,3000,100):
  model_rf = RandomForestRegressor(n_estimators=i, oob_score=True, random_state=100)
  model_rf.fit(X_train, y_train)
  pred_test_rf= model_rf.predict(X_test)
  k = r2_score(y_test, pred_test_rf)
  if k>0:
    print("------------------------------------------------------")
    print(k,i)"""

In [ ]:
prim_plot(y_train,pred_train) #[blue]
prim_plot(y_test,pred_test)   #[Yollow]

#Polynomial Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=4)
X_poly_train = poly.fit_transform(X_train)

poly.fit(X_poly_train, y_train)
lin2 = LinearRegression()
lin2.fit(X_poly_train, y_train)





print("Training")
pred_train= lin2.predict(X_poly_train)
res(y_train,pred_train)

print("Testing")
X_poly_test = poly.fit_transform(X_test)
pred_test= lin2.predict(X_poly_test)
res(y_test,pred_test)

In [ ]:
prim_plot(y_train,pred_train) #[blue]
prim_plot(y_test,pred_test)   #[Yollow]

#Kernel Ridge

In [ ]:
from sklearn.kernel_ridge import KernelRidge
krr = KernelRidge(alpha=1.0)
krr.fit(X_train, y_train)


print("Training")
pred_train= krr.predict(X_train)
res(y_train,pred_train)

print("Testing")
pred_test= krr.predict(X_test)
res(y_test,pred_test)

In [ ]:
prim_plot(y_train,pred_train) #[blue]
prim_plot(y_test,pred_test)   #[Yollow]

#SVM

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_train, y_train)

print("Training")
pred_train= regr.predict(X_train)
res(y_train,pred_train)

print("Testing")
pred_test= regr.predict(X_test)
res(y_test,pred_test)

In [ ]:
prim_plot(y_train,pred_train) #[blue]
prim_plot(y_test,pred_test)   #[Yollow]

#Lasso

In [ ]:
from sklearn import linear_model
regr = linear_model.Lasso(alpha=0.1)
regr.fit(X_train, y_train)

print("Training")
pred_train= regr.predict(X_train)
res(y_train,pred_train)

print("Testing")
pred_test= regr.predict(X_test)
res(y_test,pred_test)

In [ ]:
prim_plot(y_train,pred_train) #[blue]
prim_plot(y_test,pred_test)   #[Yollow]

#ANN Regression

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(data=dataset,x=target,color="red",kde=True)
plt.axvline(x=dataset[target].mean(),ymax=0.55,color="green",linestyle='--',label="Mean")
plt.axvline(x=dataset[target].median(),ymax=0.56,color="purple",linestyle='--',label="Median")
plt.legend()
plt.title("ST-II Max-Min Difference")

In [ ]:
plt.figure(figsize = (8,6))
sns.histplot(dataset[target],kde=True,bins=40,color="red",cumulative=True)
plt.title("Cumulative of the Target Column")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

ann = Sequential()                          # Initializing the ANN
ann.add(Dense(units=6, activation="relu"))  #Adding First Hidden Layer
ann.add(Dense(units=6, activation="relu"))  # Adding Second Hidden Layer
ann.add(Dense(units=1))   # Adding Output Layer
#If we make a regression with neural networks, we do not need to add an activation function compared to classification problems

ann.compile(optimizer="adam",loss="mean_squared_error")
ann.fit(x=X_train, y=y_train, epochs=100, batch_size=64,validation_data=(X_test,y_test),callbacks=EarlyStopping(monitor='val_loss',patience=4))

#pd.DataFrame(ann.history.history)

plt.style.use("ggplot")
pd.DataFrame(ann.history.history).plot(figsize=(12,10))


In [ ]:
ann.evaluate(X_train,y_train)

In [ ]:
ann.evaluate(X_test,y_test)

In [ ]:
predictions = ann.predict(X_test)
predictions_df = pd.DataFrame(np.ravel(predictions),columns=["Predictions"])
comparison_df = pd.concat([pd.DataFrame(y_test,columns=["Real Values"]), predictions_df],axis=1)
comparison_df

In [ ]:
# here I will visualize the real test values(y_test) versus the predicted values.
plt.figure(figsize=(8,6))
sns.scatterplot(x = np.ravel(predictions),y = y_test)
plt.title("The Scatterplot of Relationship between Actual Values and Predictions")
plt.xlabel("Predictions")
plt.ylabel("Actual Values")
#It seems that our model predicts very well

In [ ]:
print("Training")
pred_train= ann.predict(X_train)
res(y_train,pred_train)

print("Testing")
pred_test= ann.predict(X_test)
res(y_test,pred_test)